# Случайные леса
__Суммарное количество баллов: 10__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [1]:
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from task import gini, entropy, gain

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [4]:
from task import DecisionTree

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [5]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.n_estimators = n_estimators
        self.random_forest = []

    def fit(self, X, y):
        for _ in range(self.n_estimators):
            self.random_forest.append(DecisionTree(X, y,
                                                   criterion=self.criterion,
                                                   max_depth=self.max_depth,
                                                   min_samples_leaf=self.min_samples_leaf,
                                                   max_features=self.max_features))

    def predict(self, X):
        predicted_matrix = []
        for tree in self.random_forest:
            predicted_matrix.append(tree.predict(X))

        predicted_matrix = np.array(predicted_matrix)
        unique_values, indexes = np.unique(predicted_matrix, return_inverse=True)
        return unique_values[
            np.argmax(
                np.apply_along_axis(np.bincount, 0, indexes.reshape(predicted_matrix.shape), None, np.max(indexes) + 1),
                axis=0)]

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [6]:
def count_accuracy(y, y_pred):
    return np.mean(y == y_pred)

def tree_feature_importance(tree):
    X = tree.X_out_bag
    y = tree.y_out_bag
    y_pred = tree.predict(X)
    accuracy = count_accuracy(y, y_pred)
    importance = []

    for i in range(X.shape[1]):
        X_shuffle = copy.deepcopy(X)
        np.random.shuffle(X_shuffle[:, i])

        y_pred_shuffle = tree.predict(X_shuffle)
        accuracy_shuffle = count_accuracy(y, y_pred_shuffle)
        importance.append(accuracy - accuracy_shuffle)

    return np.array(importance)

def feature_importance(rfc):
    importance_matrix = []
    for tree in rfc.random_forest:
        importance_matrix.append(tree_feature_importance(tree))

    return np.mean(np.array(importance_matrix), axis=0)

def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [7]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [-1.24542340e-03 -1.29356867e-03  1.78894568e-01  1.66369467e-01
  3.37730586e-01 -2.62198624e-04]


### Задание 4 (3 балла)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

Оценка:
1. 1 балл за исправно работающий код
2. +1 балл за точность предсказания возростной группы выше 65%
3. +1 балл за точность предсказания пола выше 75%

In [8]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [9]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [10]:
from task import rfc_age

rfc_age.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc_age.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc_age), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.6582597730138714
Most important features:
1. ovsyanochan
2. rhymes
3. mudakoff
4. 4ch
5. iwantyou
6. dayvinchik
7. styd.pozor
8. pozor
9. pixel_stickers
10. pravdashowtop
11. top_screens
12. memeboizz
13. pustota_diary
14. rapnewrap
15. girlmeme
16. femalemem
17. reflexia_our_feelings
18. bot_maxim
19. i_d_t
20. tumblr_vacuum


#### Пол

In [11]:
from task import rfc_gender

rfc_gender = RandomForestClassifier(n_estimators=10)
rfc_gender.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc_gender.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc_gender), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.8423707440100883
Most important features:
1. 40kg
2. girlmeme
3. zerofat
4. modnailru
5. i_d_t
6. 9o_6o_9o
7. be.women
8. rapnewrap
9. mudakoff
10. sh.cook
11. 4ch
12. reflexia_our_feelings
13. recipes40kg
14. thesmolny
15. cook_good
16. psy.people
17. woman.blog
18. bot_maxim
19. combovine
20. beauty


### CatBoost
В качестве аьтернативы попробуем CatBoost. 

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [29]:
X, y = synthetic_dataset(1000)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

catboost_synthetic = CatBoostClassifier(loss_function='MultiClass',
                                        iterations=10,
                                        learning_rate=0.1,
                                        depth=4)
catboost_synthetic.fit(X_train, y_train)

print("Accuracy:", np.mean(catboost_synthetic.predict(X_test) == y_test))

0:	learn: 0.9255685	total: 2.1ms	remaining: 18.9ms
1:	learn: 0.7872102	total: 4.66ms	remaining: 18.6ms
2:	learn: 0.6825833	total: 6.48ms	remaining: 15.1ms
3:	learn: 0.5969569	total: 8.35ms	remaining: 12.5ms
4:	learn: 0.5255767	total: 10.2ms	remaining: 10.2ms
5:	learn: 0.4652426	total: 12ms	remaining: 8.02ms
6:	learn: 0.4114182	total: 13.8ms	remaining: 5.91ms
7:	learn: 0.3652614	total: 15.6ms	remaining: 3.89ms
8:	learn: 0.3253697	total: 17.3ms	remaining: 1.92ms
9:	learn: 0.2923496	total: 19.1ms	remaining: 0us
Accuracy: 0.3344888888888889


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

Оценка:
1. 1 балл за исправно работающий код
2. +1 балл за точность предсказания возростной группы выше 65%
3. +1 балл за точность предсказания пола выше 75%

In [14]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

In [15]:
catboost_age = CatBoostClassifier(loss_function='MultiClass',
                                  iterations=10,
                                  learning_rate=1,
                                  depth=2)
catboost_age.fit(X_train, y_age_train)
catboost_age.save_model('catboost_age.cbm', format='cbm')

0:	learn: 1.0021309	total: 169ms	remaining: 1.52s
1:	learn: 0.9359616	total: 175ms	remaining: 701ms
2:	learn: 0.8973860	total: 179ms	remaining: 418ms
3:	learn: 0.8678255	total: 183ms	remaining: 275ms
4:	learn: 0.8414548	total: 188ms	remaining: 188ms
5:	learn: 0.8233869	total: 192ms	remaining: 128ms
6:	learn: 0.8079535	total: 196ms	remaining: 84ms
7:	learn: 0.7910472	total: 200ms	remaining: 50ms
8:	learn: 0.7803472	total: 203ms	remaining: 22.6ms
9:	learn: 0.7665295	total: 205ms	remaining: 0us


In [16]:
catboost_gender = CatBoostClassifier(loss_function='MultiClass',
                                  iterations=10,
                                  learning_rate=1,
                                  depth=2)
catboost_gender.fit(X_train, y_sex_train)
catboost_gender.save_model('catboost_gender.cbm', format='cbm')

0:	learn: 0.6139866	total: 1.99ms	remaining: 17.9ms
1:	learn: 0.5689612	total: 5.31ms	remaining: 21.3ms
2:	learn: 0.5379372	total: 7.04ms	remaining: 16.4ms
3:	learn: 0.5165357	total: 8.76ms	remaining: 13.1ms
4:	learn: 0.4952735	total: 10.5ms	remaining: 10.5ms
5:	learn: 0.4729781	total: 12.2ms	remaining: 8.11ms
6:	learn: 0.4562936	total: 13.9ms	remaining: 5.95ms
7:	learn: 0.4438425	total: 15.6ms	remaining: 3.91ms
8:	learn: 0.4321296	total: 17.4ms	remaining: 1.93ms
9:	learn: 0.4239501	total: 20ms	remaining: 0us


#### Возраст

In [19]:
print("Accuracy:", np.mean(catboost_age.predict(X_test) == y_age_test))

Accuracy: 0.331483392674553


#### Пол

In [20]:
print("Accuracy:", np.mean(rfc_gender.predict(X_test) == y_sex_test))

Accuracy: 0.89281210592686
